# Lab 01 - Histograms manipulations and Filtering

Computer Vision 2023-24 (M. Caligiuri, P. Zanuttigh, F. Lincetto)

In this lab you will explore two different topics:

1. Histogram-based processing:
    - Histogram equalization,
    - Histogram matching;
2. Filtering for denoising:
    - Gaussian filter,
    - Median filter,
    - Bilateral filter.

---

## Import the necessary libraries

In [ ]:
#author: Risi Emilio 
#matr :  2122841

# Import the necessary packages

import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

---

## Histogram equalization

In this section you will have to perform the histogram equalization of the image `./data/barbecue.jpg`. Do that using first the **BGR** color space and then the **LAB** color space. You will see the difference in the results.

### Auxiliary functions

Write here all the auxiliary functions you will need.
More precisely you have to implement the following functions:
- `calc_hist` -> given in input an image it will return a list containing the histogram of each channel,
- `show_histogram` -> given in input the list of all the histograms of an image it will return an image containing the plot of the histograms of each channel side by side (use matplotlib to generate the plot than convert them to image using `cv2`),

  ![hists_example](data/hists.png)
- `img_hist` -> given in input an image and the correspondent list of histograms it will return the image with the histograms plotted on top of it (call `show_histogram`).

  ![img_hist_example](data/img_hists.png)

In [ ]:
# Define a function to compute the histogram of the image (channel by channel)
def calc_hists(img: np.ndarray) -> list:
    """
    Calculates the histogram of the image (channel by channel).

    Args:
        img (numpy.ndarray): image to calculate the histogram
    
    Returns:
        list: list of histograms
    """

    # Add your code here
    hist_r = cv.calcHist([img], [0], None, [256], [0, 256])
    hist_g = cv.calcHist([img], [1], None, [256], [0, 256])
    hist_b = cv.calcHist([img], [2], None, [256], [0, 256])
    return hist_r, hist_g, hist_b

In [ ]:
# Define a function to plot together the three histograms of the image
# (plot only one if it is grayscale)
def show_histogram(hists: list) -> np.ndarray:
    """
    Shows the histogram of the image.
    
    Args:
        hists (list): list of histograms to plot

    Returns:
        img (numpy.ndarray): image containing the histograms
     """   
    
    
    # Add your code here to generate the matplotlib figurecontaining the histograms
    colors = [ "Blue","Green","Red"]

    fig, axs = plt.subplots(1, 3, figsize=(40, 8))  
    
    for i in range(3):
        axs[i].plot(hists[i], color=colors[i],linewidth = 3)  
        axs[i].tick_params(labelsize=40)  
        axs[i].set_xlabel('Bins', fontsize=40)  
        axs[i].set_ylabel('# of Pixels', fontsize=40)    
        axs[i].set_title(f'{colors[i]} Histogram', fontsize=40)
         # Set the axis to have 0 at the center
        axs[i].spines['left'].set_position(('data', 0))
       
       
    
        

    # Convert figure to canvas

    plt.tight_layout()
    
    canvas = plt.get_current_fig_manager().canvas
    plt.close()

    # Render the canvas
    canvas.draw()

    # Convert canvas to image
    img = np.frombuffer(canvas.buffer_rgba(), dtype='uint8')
    img = img.reshape(canvas.get_width_height()[::-1] + (4,))

    # Convert image to cv2 format
    return cv.cvtColor(img, cv.COLOR_RGB2BGR)

In [ ]:
# Define a function to fuse the histogram image with the original one
def img_hist(img: np.ndarray, hists: list[np.ndarray], scaling_factor: float = 0.3) -> np.ndarray:
    """
    Fuse the image with the histogram.
    
    Args:
        img (np.ndarray): image
        hist (list(np.ndarray)): histograms of the image
        
    Returns:
        img (numpy.ndarray): image containing the original image with the histograms on top
    """
    # Add your code here
    img_h = show_histogram(hists)
    #match the histogram width
    width = img_h.shape[1]
    h_w = img.shape[0]/img.shape[1]
    height = int(  h_w * width)
    resized_img = cv.resize(img, (width, height))
    s_image = np.vstack(( img_h,resized_img))
    
    
    #scale resize
    new_width = int(s_image.shape[1] * scaling_factor)
    new_height = int(s_image.shape[0] * scaling_factor)
    resized_img = cv.resize(s_image, (new_width, new_height))
        
    
    
    return resized_img
    
    
    

### Equalize the image in the BGR space

Equalize the image (`./data/barbecue.jpg`) in the BGR space and show the result.

In [ ]:
# Load image and and compute its histogram
# Add your code here
#better view
plt.rcParams['figure.dpi'] = 300

img_bgr = cv.imread('./data/barbecue.jpg')
#histo bgr
hD_bgr = calc_hists(img_bgr)
#histo image bgr
hD_img = show_histogram(hD_bgr)
#image with histo
img_hi_bgr = img_hist(img_bgr,hD_bgr)


In [ ]:
# Show the image with its histogram before the equalization
# # Add your code here
plt.axis('off')  # Turn off axis labels and ticks
plt.imshow(cv.cvtColor(img_hi_bgr, cv.COLOR_BGR2RGB))


In [ ]:
# Equalize the histogram of the image
#convert to yuv
img_yuv = cv.cvtColor(img_bgr, cv.COLOR_BGR2YUV)
# equalize the histogram of the Y channel
img_yuv[:,:,0] = cv.equalizeHist(img_yuv[:,:,0])
# convert the YUV image back to RGB format
img_eq_bgr = cv.cvtColor(img_yuv, cv.COLOR_YUV2BGR)

img_hi_eq_bgr = img_hist(img_eq_bgr,calc_hists(img_eq_bgr)) 

# Add your code here
# Show the image and the relative histogram (in the same window)
plt.axis('off')  # Turn off axis labels and ticks
plt.imshow(cv.cvtColor(img_hi_eq_bgr, cv.COLOR_BGR2RGB))

# Add your code here

### Equalize the image in the LAB space

Equalize the image (`./data/barbecue.jpg`) in the LAB space and show the result.

In [ ]:
# Load the image and convert it to LAB colorspace
#this is BGR blue green red
img_bgr = cv.imread('./data/barbecue.jpg')
img_lab = cv.cvtColor(img_bgr, cv.COLOR_BGR2LAB)

# Add your code here

In [ ]:
# Equalize the histogram of the L channel


l_channel, a_channel, b_channel = cv.split(img_lab)
# Apply histogram equalization to the L channel
l_channel_eq = cv.equalizeHist(l_channel)
img_eq_lab = cv.merge((l_channel_eq, a_channel, b_channel))

# Convert the equalized Lab image back to BGR
img_eq_bgr = cv.cvtColor(img_eq_lab, cv.COLOR_LAB2BGR)
# Extract the histograms for each channel

# Add your code here
img_hi_eq_bgr = img_hist(img_eq_bgr,calc_hists(img_eq_bgr))




In [ ]:
# Show the image and its relative histogram
plt.axis('off')  # Turn off axis labels and ticks
plt.imshow(cv.cvtColor(img_hi_eq_bgr, cv.COLOR_BGR2RGB))
# Add your code here

---

## Histogram matching

In this section you will have to perform the histogram matching of the image `./data/panorama_left.jpg` to the image `./data/panorama_right.jpg`. Do that in the **BGR** color space.

### Histogram matching function

The function to perform the histogram matching is provided here. The function takes in input the image to be matched and the image to match and will return the matched image.

In [ ]:
#  function to perform the histogram matching
def hist_match(source: np.ndarray, reference: np.ndarray) -> np.ndarray:
    """
    Adjust the pixel values of a color image such that its histogram
    matches that of a target one.

    Args:
        source (numpy.ndarray): Image to transform; the histogram is computed over the flattened array
        reference (numpy.ndarray): Template image; can have different dimensions from source
    Returns:
        numpy.ndarray: The transformed output image
    """
    
     # Assert that the images have the same number of channels (grayscale or RGB) and the same dimensions
    assert source.shape[2] == reference.shape[2], "Images must have the same number of channels"
    assert source.shape[:2] == reference.shape[:2], "Images must have the same dimensions"

    # Compute the source image's histogram and CDF
    src_hists = [np.histogram(source[..., i].flatten(), 256, [0,256])[0] for i in range(source.shape[2])]
    src_cdfs = [hist.cumsum() for hist in src_hists]
    src_cdfs_normalized = [cdf / float(cdf.max()) for cdf in src_cdfs]
 
    # Compute the reference image's histogram and CDF
    ref_hists = [np.histogram(reference[..., i].flatten(), 256, [0,256])[0] for i in range(reference.shape[2])]
    ref_cdfs = [hist.cumsum() for hist in ref_hists]
    ref_cdfs_normalized = [cdf / float(cdf.max()) for cdf in ref_cdfs]

    # Create a lookup table to map pixel values from the source to the reference
    lookup_tables = [np.zeros(256) for _ in range(source.shape[2])]
    lookup_values = [0] * source.shape[2]
    for index in range(len(lookup_tables)):
        for src_pixel_val in range(len(src_cdfs_normalized[index])):
            lookup_values[index]
            for ref_pixel_val in range(len(ref_cdfs_normalized[index])):
                if ref_cdfs_normalized[index][ref_pixel_val] >= src_cdfs_normalized[index][src_pixel_val]:
                    lookup_values[index] = ref_pixel_val
                    break
            lookup_tables[index][src_pixel_val] = lookup_values[index]

    # Apply the lookup table to the source image
    matched = np.stack([cv.LUT(source[..., i], lookup_tables[i]).astype(np.uint8) for i in range(len(lookup_tables))], axis=-1)

    return matched

### Match the images

In [ ]:
# Load the two iamges

# Add your code here
imgl_bgr = cv.imread("./data/panorama_left.jpg")
imgr_bgr = cv.imread("./data/panorama_right.jpg")



In [ ]:
# Visualize the two images with the relative histograms
# before the histogram matching

#left histo
imgl_hi_bgr = img_hist(imgl_bgr,calc_hists(imgl_bgr))
#right histo
imgr_hi_bgr = img_hist(imgr_bgr,calc_hists(imgr_bgr))


# Add your code here

plt.figure()
plt.imshow(cv.cvtColor(imgl_hi_bgr, cv.COLOR_BGR2RGB))
plt.axis('off')  

plt.figure()
plt.imshow(cv.cvtColor(imgr_hi_bgr, cv.COLOR_BGR2RGB))
plt.axis('off') 





# Add your code here

In [ ]:
# Match the histograms of the two images

# Add your code here
imgl_m_bgr = hist_match(imgl_bgr,imgr_bgr)
imgr_m_bgr = hist_match(imgr_bgr,imgl_bgr)

#left histo
imgl_m_hi_bgr = img_hist(imgl_m_bgr,calc_hists(imgl_m_bgr))
#right histo
imgr_m_hi_bgr = img_hist(imgr_m_bgr,calc_hists(imgr_m_bgr))


# Visualize the two iamges with the relative histograms

plt.figure()
plt.imshow(cv.cvtColor(imgl_m_hi_bgr, cv.COLOR_BGR2RGB))
plt.axis('off')  

plt.figure()
plt.imshow(cv.cvtColor(imgr_m_hi_bgr, cv.COLOR_BGR2RGB))
plt.axis('off') 





# Add your code here

---

## Image denoising

In this section you will have to perform the denoising of the image ./data/overexposed.jpg. Do that using the following filters:

    Gaussian filter,
    Median filter,
    Bilateral filter.

For each filter you will have to try different values for the parameters in order to see the effect of each parameter on the result. To do that in an interactive way it is required that you implement the cv.trackbar function. In this way you will be able to change the parameters of the filters and to see the result in real time.

In order to build a better code you have to implement each filter as a class (child of a main filter class called Filter). The structure to do this is already provided in this notebook

### Define the filters classes

In [ ]:
# Define the main Filter class with the common methods
# (this class will be used as a base class for the other filters)
class Filter:
    def __init__(self, size: int):
        # Constructor of the class
        if size % 2 == 0:
            size += 1
        self.set_filter_size(size)
    
    def set_filter_size(self, size: int) -> None:
        # Method to set the filter size
        if size % 2 == 0:
            size += 1
        self.filter_size = size
    
    def get_filter_size(self) -> int:
        # Method to get the filter size
        return self.filter_size

    def __call__(self, image: np.ndarray) -> None:
        # Method to perform the filtering
        # (it must be implemented in the derived classes)
        raise NotImplementedError

In [ ]:
# Define the class for the gaussian filter
class GaussianFilter(Filter):
    def __init__(self, size: int, sigma_g: float):
        super().__init__(size)
        self.sigma = sigma_g

    def set_sigma(self, s: float) -> None:
        # Method to set the sigma value

        # Add your code here (remove the pass statement)
        self.sigma = s 

    def get_sigma(self) -> float:
        # Method to get the sigma value
        
        # Add your code here (remove the pass statement)
        return self.sigma
    
    def __call__(self, image: np.ndarray, size: int, sigma: float) -> None:
        # Method to call the filter (it is called when the object is called)
        # This function will be called by the trackbar object and will update the imamge to show
        
        # Add your code here (remove the pass statement)
        
        self.set_filter_size(size)
        self.set_sigma(sigma)

        return cv.GaussianBlur(image, (self.get_filter_size(),self.get_filter_size()), self.sigma)



# Define the class for the median filter
class MedianFilter(Filter):
    def __init__(self, size: int):
        super().__init__(size)

    def __call__(self, image: np.ndarray, size: int) -> None:
        # Method to call the filter (it is called when the object is called)
        # This function will be called by the trackbar object and will update the image to show
        
        # Add your code here (remove the pass statement)
        self.set_filter_size(size)
        return cv.medianBlur(image, self.get_filter_size())


# Define the class for the bilateral filter
class BilateralFilter(Filter):
    def __init__(self, size: int, sigma_s: float, sigma_r: float):
        super().__init__(size)
        self.sigma_space = sigma_s
        self.sigma_range = sigma_r

    def set_sigma_range(self, sr: float) -> None:
        # Method to set the sigma range value

        # Add your code here (remove the pass statement)
        self.sigma_range = sr

    def get_sigma_range(self) -> float:
        # Method to get the sigma range value
        
        # Add your code here (remove the pass statement)
        return self.sigma_range


    def set_sigma_space(self, ss: float) -> None:
        # Method to set the sigma space value

        # Add your code here (remove the pass statement)
        self.sigma_space = ss


    def get_sigma_space(self) -> float:
        # Method to get the sigma space value
        
        # Add your code here (remove the pass statement)
        return self.sigma_space
    
    def __call__(self, image: np.ndarray, sigma_s: float, sigma_r: float) -> None:
        # Method to call the filter (it is called when the object is called)
        # This function will be called by the trackbar object and will update the imamge to show
        
        # Add your code here (remove the pass statement)
        self.set_sigma_space(sigma_s)        
        self.set_sigma_range(sigma_r)        
        return cv.bilateralFilter(image, self.get_filter_size(), self.sigma_range, self.sigma_space)


### Load the image and initialize the filters objects

In [ ]:
# Load and show the image to filter
img_bgr = cv.imread("./data/overexposed.jpg")

# Add your code here

In [ ]:
# Initialize the filters

# Add your code here
gaussian_filter = GaussianFilter(size=5, sigma_g=1.5)
median_filter = MedianFilter(size=5)
bilateral_filter = BilateralFilter(size=9, sigma_s=75, sigma_r=75)


### Test the filters

In [ ]:
# MEDIAN BLUR FILTER
#TO DO CHANGE START VALUE TRACKBAR


# Create a window for the trackbars
#need to change for the use with objects
def on_trackbar(val):
    kernel_size = max(1, val * 2 + 1)
    filtered_image = median_filter(img_bgr,kernel_size)
    cv.imshow('Median Blur Filter', filtered_image)

# Add your code here

cv.namedWindow('Median Blur Filter')

# Create the trackbar

initial_kernel_size = median_filter.get_filter_size()


cv.createTrackbar('Kernel Size', 'Median Blur Filter', initial_kernel_size, 20, on_trackbar)

cv.waitKey(0)
cv.destroyAllWindows()



filtered_image = median_filter(img_bgr,median_filter.get_filter_size())


# Add your code here


# Show the filtered image with a slider to change the kernel size
cv.imshow('Original Image', img_bgr)
cv.imshow('Median Blur Filter', filtered_image)


cv.waitKey(0)
cv.destroyAllWindows()

# Add your code here

In [ ]:
# GAUSSIAN BLUR FILTER


# Create a window for the trackbars


# Add your code here
def on_trackbar(val):
    kernel_size = max(1, val * 2 + 1)
    
    filtered_image = gaussian_filter(img_bgr,kernel_size,gaussian_filter.get_sigma())
    cv.imshow('Gaussian Filter', filtered_image)

cv.namedWindow('Gaussian Filter')


# Create the trackbar
initial_kernel_size = gaussian_filter.get_filter_size()

cv.createTrackbar('Kernel Size', 'Gaussian Filter', initial_kernel_size, 20, on_trackbar)
filtered_image = gaussian_filter(img_bgr,gaussian_filter.get_filter_size(),gaussian_filter.get_sigma())

cv.waitKey(0)
cv.destroyAllWindows()


# Add your code here


# Show the filtered image with a slider to change the kernel size

# Show the filtered image with a slider to change the kernel size
cv.imshow('Original Image', img_bgr)
cv.imshow('Gaussian Filter', filtered_image)


cv.waitKey(0)
cv.destroyAllWindows()

# Add your code here

In [ ]:
# GAUSSIAN BLUR FILTER


# Create a window for the trackbars


# Add your code here
def on_trackbar(val):
    kernel_size = max(1, val * 2 + 1)
    bilateral_filter.set_filter_size(kernel_size)
    filtered_image = bilateral_filter(img_bgr,bilateral_filter.get_sigma_space(),bilateral_filter.get_sigma_range())
    cv.imshow('Bilateral Filter', filtered_image)

cv.namedWindow('Bilateral Filter')


# Create the trackbar
initial_kernel_size = bilateral_filter.get_filter_size()

cv.createTrackbar('Kernel Size', 'Bilateral Filter', initial_kernel_size, 20, on_trackbar)
filtered_image = bilateral_filter(img_bgr,bilateral_filter.get_sigma_space(),bilateral_filter.get_sigma_range())

cv.waitKey(0)
cv.destroyAllWindows()


# Add your code here


# Show the filtered image with a slider to change the kernel size

# Show the filtered image with a slider to change the kernel size
cv.imshow('Original Image', img_bgr)
cv.imshow('Bilateral Filter', filtered_image)


cv.waitKey(0)
cv.destroyAllWindows()

# Add your code here